In [88]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from natsort import natsorted

In [27]:
def detect_weld(image, padding=0):
    try:
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
        image = np.array(image, dtype=np.float32)
        image = cv2.GaussianBlur(image, (31,31), 0)

        for i in range(9):
            image = np.split(image, 2, axis=0)
            image = np.add(image[0], image[1])
            
        image = np.gradient(np.squeeze(image))
        y1, y2 = 512 - int(np.argmin(image)) , 512 - int(np.argmax(image))
    
        
        return (y1 - padding, y2 + padding)
    except:
        return -1, -1

In [94]:
def image_centering(image, padding = 0):
    #원래 이미지 사이즈
    h, w = image.shape
    #용접부의 중심점
    y1, y2 = detect_weld(image, padding=padding)
    
    
    # 용접부 상단, 용접부, 용접부 하단으로 나누기
    top = image[:y1]
    weld = image[y1:y2]
    bottom = image[y2:]
    
    weld_center = weld.shape[0]//2
    #top을 y가 512가 되도록 flip하면서 padding
    top = np.flip(top, axis=0)
    top = np.pad(top, ((0, 512 - top.shape[0]), (0,0)), 'reflect')
    top = np.flip(top, axis=0)
    
    top = cv2.GaussianBlur(top, (3,3), 0)
    bottom = np.pad(bottom, ((0, 512 - bottom.shape[0]), (0,0)), 'reflect')
    bottom = cv2.GaussianBlur(bottom, (3,3), 0)
    
    #세로로 합치기
    concat_img = np.concatenate((top, weld, bottom), axis=0)
    
    #용접부 중심 기준 원래 사이즈로 crop
    weld_center = weld_center + 512
    crop_img = concat_img[weld_center - h//2 : weld_center + h//2, :]

    
    

    return crop_img
    

In [97]:
image_list = glob("/home/dais01/VirtualFlaw/RT_Project/src/Archive/**/*.tif", recursive=True)

for image in image_list:
    img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    try:
        img = image_centering(img, padding=15)
        cv2.imwrite(image.replace("Archive", "Archive_centering").replace(".tif", ".png"), img, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    except:
        print(image)
    
    

/home/dais01/VirtualFlaw/RT_Project/src/Archive/2/img_#2_71169_1.tif
/home/dais01/VirtualFlaw/RT_Project/src/Archive/1/img_#1_513_1.tif


True